# <strong>  0. Abstract </strong>

Training Deep Neural Networks is complicated by the fact
that the distribution of each layer’s inputs changes during
training, as the parameters of the previous layers change.
This slows down the training by requiring lower learning
rates and careful parameter initialization, and makes it no
-
toriously hard to train models with saturating nonlinearities. We refer to this phenomenon as internal covariate
shift, and address the problem by normalizing layer inputs. Our method draws its strength from making normalization a part of the model architecture and performing the
normalization for each training mini-batch. Batch Normalization allows us to use much higher learning rates and
be less careful about initialization. It also acts as a regularizer, in some cases eliminating the need for Dropout.
Applied to a state-of-the-art image classification model,
Batch Normalization achieves the same accuracy with 14
times fewer training steps, and beats the original model
by a significant margin. Using an ensemble of batchnormalized networks, we improve upon the best published
result on ImageNet classification: reaching 4.9% top-5
validation error (and 4.8% test error), exceeding the accuracy of human raters

이전 층의 파라미터가 변화로 인해 매번 훈련마다 각 layer의 입력값의 distribution이 변하여 DNN을 훈련하는 것은 매우 복잡하다. 위 문제는 **더 작은 learning rate와 파라미터 초기화의 세심함을 요구**하여 훈련을 느리게 하며 포화 비선형 훈련 모델을 학습하기 어렵게 하는데 악명이 높다. 
우리는 이러한 현상을 **internal covariate shift**라고 부르며 층의 입력을 표준화 함으로써 문제를 해결했다.
우리는 모델 구성의 일부에서 표준화를 진행하고 각 훈련용 mini-batch를 표준화하여 해결법의 강점을 끌어냈다. **Batch Normalization**은 **더 큰 learning rate와 초기화에 대한 부주의함을 허락한다.** Batch Normalization은 **regularizer로도 작동**하여 경우에 따라 Dropout의 필요성을 제거해준다. 최첨단 이미지 분류 모델에 Batch Normalization을 적용해보면 14번의 더 적은 training step을 사용하여 같은 정확도에 도달하고 기존 모델의 상당한 편차(차이?)를 통제한다. 우리는 표준화된 네트워크의 앙상블을 사용하여 발표된 최선의 ImageNet 분류 결과를 더 개선했다: top-5 validation error는 4.9%(test error는 4.8%)에 도달했고, 사람의 정확도를 넘어선 결과다.

**<정리>**

Batch Normalization을 사용하면 다음과 같은 장점을 가진다.

* 학습을 빠르게 진행할 수 있다. (큰 learning rate 사용 가능)
* 초깃값에 크게 의존하지 않는다. 
* overfitting을 억제한다. (드롭아웃 필요성 감소)
* 비선형 함수를 이용할때 gradient vanishing 등의 문제를 예방한다.


# <strong>  1. Introduction </strong>

Deep learning has dramatically advanced the state of the
art in vision, speech, and many other areas. Stochastic gradient descent (SGD) has proved to be an effective way of training deep networks, and SGD variants
such as momentum (Sutskever et al., 2013) and Adagrad
(Duchi et al., 2011) have been used to achieve state of the
art performance. SGD optimizes the parameters
Θ of the
network, so as to minimize the loss

$Θ = arg \, min_Θ
\frac{1}{N}
∑_{i=1}^{N}
ℓ(x_i,Θ)$

where $x_{1…N}$ is the training data set.
With SGD, the training proceeds in steps, and at each step we consider a minibatch $x_{1 \dots m}$ of size m. The mini-batch is used to approximate the gradient of the loss function with respect to the parameters, by computing

$
\frac{1}{m}
\frac{∂ℓ(x_i, Θ)}{∂Θ}
$

Deep Learning은 vision, speech 그리고 그 외 많은 최첨단 분야에서 극적인 발전을 이뤘다. SGD는 deep network를 훈련하는 효율적인 방법을 제공했으며 momentum, Adagrad와 같은 SGD의 변형들은 최신 기술 성능을 달성하는데 사용되어 왔다. SGD는 $x_{1…N}$가 train 데이터셋인 loss값을 최소화하기 위해 네트워크의 파라미터들($Θ$)을 최적화한다.

SGD를 이용해 단계적으로 train 과정을 지속했고 각 단계에서 크기가 m인 미니배치 $x_{1 \dots m}$를 사용했다. 미니배치는 아래 식을 계산함으로써 파라미터에 대한 loss function의 gradient를 근사하는데 사용된다.

Using mini-batches of examples, as opposed to one example at a time, is helpful in several ways. First, the gradient
of the loss over a mini-batch is an estimate of the gradient
over the training set, whose quality improves as the batch
size increases. Second, computation over a batch can be
much more efficient than
m computations for individual
examples, due to the parallelism afforded by the modern
computing platforms.
While stochastic gradient is simple and effective, it
requires careful tuning of the model hyper-parameters,
specifically the learning rate used in optimization, as well
as the initial values for the model parameters. The training is complicated by the fact that the inputs to each layer
are affected by the parameters of all preceding layers – so
that small changes to the network parameters amplify as
the network becomes deeper.
The change in the distributions of layers’ inputs
presents a problem because the layers need to continuously adapt to the new distribution. When the input distribution to a learning system changes, it is said to experience covariate shift (Shimodaira, 2000). This is typically
handled via domain adaptation (Jiang, 2008). However,
the notion of covariate shift can be extended beyond the
learning system as a whole, to apply to its parts, such as a
sub-network or a layer. Consider a network computing

$ℓ= F_2(F_1(u, Θ_1), Θ_2)$

where
$F_1$ and
$F_2$ are arbitrary transformations, and the
parameters
$Θ_1, Θ_2$ are to be learned so as to minimize
the loss
$ℓ$. Learning
$Θ_2$ can be viewed as if the inputs
$x =F_1(u, Θ_1)$ are fed into the sub-network

$ℓ= F_2(x, Θ_2)$

For example, a gradient descent step

$Θ_2 ← Θ_2 - \frac{α}{m}\sum_{i=1}^{m} \frac{∂F_2(x_i, Θ_2)}{∂Θ_2}$

(for batch size
m and learning rate
α) is exactly equivalent
to that for a stand-alone network
F2 with input
x. Therefore, the input distribution properties that make training
more efficient – such as having the same distribution between the training and test data – apply to training the
sub-network as well. As such it is advantageous for the
distribution of
x to remain fixed over time. Then,
$Θ_2$ does not have to readjust to compensate for the change in the distribution of x.


example들의 미니배치들을 사용하는 것은 한 번에 하나의 example을 사용하는 것과 달리 몇 가지 방면에서 유익하다. 첫 번째로 미니배치에 대한 loss의 gradient는 배치 크기가 클 수록 성능이 증가하는 train 데이터셋에 대한 gradient 값의 추정치이다. 현대의 계산 플랫폼이 제공하는 병렬처리 덕분에 두 번째로 배치에 대한 계산은 각각의 example에 대해 m개의 계산을 수행하는 것보다 효율적이다. 

**stochastic gradient는 간단하고 효과적이지만 최적화에 사용되는 learning rate 등의 hyper-parameter들과 모델 파라미터의 초깃값에 세심한 tuning이 필요하다.** 각 layer의 입력은 이전 층의 모든 파라미터들에게 영향을 받기 때문에 훈련은 복잡하다. - 특히 네트워크가 깊을수록 네트워크 파라미터들의 작은 변화는 복잡함을 증폭시킨다. layer들의 입력의 distribution에서의 변화는 layer가 끊임없이 새로운 distribution에 적응해야하므로 문제가 된다. 학습 시스템에 대한 입력의 distribution이 바뀌는 것을 **covariate shift를 경험했다**고 표현한다. 이것은 전형적으로 domain adaptation을 통해 다뤄진다. 하지만 covariate shift의 개념은 학습 시스템 전체로 확장되며 sub-network 혹은 층과 같이 일부에 적용된다. 아래 식에서 $F1, F2$는 임의의 transformation이고, $Θ_1, Θ_2$는 손실 $ℓ$를 최소화하가 위해 학습된다. 

$Θ_2$를 학습하는 것은 입력을 $ x= F_1(u, Θ_1)$이 아래의 sub-network에 반영된 것 처럼 볼 수 있다.

예를 들어 한 번의 gradient descent 단계(batch size가 m이고 learning rate는 α)는 입력이 x인 $F_2$ network 한 개가 있는 것과 동일하다. **그러므로 training과 test 데이터의 distiribution이 같게 주어진다면 더 효과적인 훈련을 수행하는 입력 distribution의 특징은 sub-network에 동일하게 적용된다.** 즉, x의 distribution을 계속 고정하는 것은 유익하다. 그러면 $Θ_2$는 더 이상 x의 distribution에서의 변화를 comensate(보상하다)하는데 조정할 필요가 없다.




Fixed distribution of inputs to a sub-network would
have positive consequences for the layers outside the subnetwork, as well. Consider a layer with a sigmoid activation function $z = g(Wu + b)$ where u is the layer input,
the weight matrix W and bias vector b are the layer parameters to be learned, and 
$g(x) = \frac{1}{1+exp^{−x}}$
. As $|x|$increases, $g'(x)$ tends to zero. This means that for all dimensions of $x = Wu+b$ except those with small absolute
values, the gradient flowing down to u will vanish and the
model will train slowly. However, since x is affected by
W, b and the parameters of all the layers below, changes
to those parameters during training will likely move many
dimensions of x into the saturated regime of the nonlinearity and slow down the convergence. This effect is
amplified as the network depth increases. In practice,
the saturation problem and the resulting vanishing gradients are usually addressed by using Rectified Linear Units
(Nair & Hinton, 2010) ReLU(x) = max(x, 0), careful
initialization (Bengio & Glorot, 2010; Saxe et al., 2013),
and small learning rates. If, however, we could ensure
that the distribution of nonlinearity inputs remains more
stable as the network trains, then the optimizer would be
less likely to get stuck in the saturated regime, and the
training would accelerate.


또한 sub-network에 대한 입력의 고정된 distribution은 subnetwork 밖에 있는 층에도 긍정적인 결과를 가져온다. sigmoid 활성화 함수로 이루어진 층 $z = g(Wu + b)$(u는 층의 입력, 가중치 행렬 W, 편향 벡터 b, $g(x) = \frac{1}{1+exp^{−x}}$)를 생각해보자. $|x|$가 증가할 때, $g'(x)$는 0으로 가능 경향이 있다. 이 현상은 절대값이 작은 차원을 제외한 $x = Wu+b$의 모든 차원들에 대해 u에 대해 하강하는 gradient가 사라지고 모델이 느리게 학습하게 한다. 하지만 x는 W, b 그리고 아래 층의 모든 파라미터에 영향을 받기 때문에 학습동안 파라미터들의 변화는 비선형의 포화 부분에 x의 많은 차원을 이동시키고 모델을 느리게 한다. 이 영향은 network가 deep할 수록 증폭된다. 실제로 포화 문제와 gradient vanishing 결과는 일반적으로 RELU, 세심한 초기화, 작은 learning rate를 이용해 해결한다. 하지만 만약 우리가 network 학습 때 비선형 입력의 distribution을 더 안정적이라 보장할 수 있다면 optimizer는 포화 부분에 덜 갇히고 훈련은 가속화 될 것이다.

We refer to the change in the distributions of internal
nodes of a deep network, in the course of training, as Internal Covariate Shift. Eliminating it offers a promise of
faster training. We propose a new mechanism, which we
call Batch Normalization, that takes a step towards reducing internal covariate shift, and in doing so dramatically accelerates the training of deep neural nets. It accomplishes this via a normalization step that fixes the
means and variances of layer inputs. Batch Normalization
also has a beneficial effect on the gradient flow through
the network, by reducing the dependence of gradients
on the scale of the parameters or of their initial values.
This allows us to use much higher learning rates without the risk of divergence. Furthermore, batch normalization regularizes the model and reduces the need for
Dropout (Srivastava et al., 2014). Finally, Batch Normalization makes it possible to use saturating nonlinearities
by preventing the network from getting stuck in the saturated modes.


학습중 deep network 내부 노드의 distribution 변화를 **Internal Covariate Shift**라고 한다. 해당 현상을 없애면 더 빠른 훈련이 가능하다. 우리는 internal covariate shift가 감소하는 방향으로 나아가면서 deep neural nets의 훈련을 가속화하는 **Batch Normalization**이라는 새로운 방법론을 제시한다. 이 방법은 층의 입력들의 평균과 분산을 고정하여 normalization 단계를 진행한다. Batch Normalization은 파라미터의 크기 혹은 파라미터의 초기값에대한 gradient의 의존성을 줄여줌으로써 network의 gradient flow에 유익한 영향을 미친다. 게다가 batch normalization은 model을 규제해주고 dropout의 필요성을 줄여준다. 마지막으로 batch normalization은 포화 상태에 network가 갇히는 것을 예방함으로써 포화 비선형을 사용하는 것을 가능하게 해준다.

In Sec. 4.2, we apply Batch Normalization to the bestperforming ImageNet classification network, and show
that we can match its performance using only 7% of the
training steps, and can further exceed its accuracy by a
substantial margin. Using an ensemble of such networks
trained with Batch Normalization, we achieve the top-5
error rate that improves upon the best known results on
ImageNet classification.


Sec. 4.2에서 우리는 가장 좋은 성적을 거둔 ImageNet 분류 network에 Batch Normalization을 적용해봤고 훈련 단계에서 우리가 단 7%의 훈련 단계를 사용하는 것과 동일한 효과를 보여주었으며 상당한 차이로 정확도를 넘어섰다. Batch Normalization을 적용한 network에 앙상블을 적용하여 ImageNet 분류 결과에서 알려진 가장 좋은 성적에서 개선된 top-5 error rate를 달성했다.

# <strong> 2. Towards Reducing Internal Covariate Shift </strong>

We define Internal Covariate Shift as the change in the
distribution of network activations due to the change in
network parameters during training. To improve the training, we seek to reduce the internal covariate shift. By
fixing the distribution of the layer inputs x as the training
progresses, we expect to improve the training speed. It has
been long known (LeCun et al., 1998b; Wiesler & Ney,
2011) that the network training converges faster if its inputs are whitened – i.e., linearly transformed to have zero
means and unit variances, and decorrelated. As each layer
observes the inputs produced by the layers below, it would
be advantageous to achieve the same whitening of the inputs of each layer. By whitening the inputs to each layer,
we would take a step towards achieving the fixed distributions of inputs that would remove the ill effects of the
internal covariate shift.

Internal Covariate Shift란 훈련중 network 파라미터들의 변화 때문에 발생하는 network activation의 distribution 변화로 정의한다. training을 개선하기 위해서 우리는 internal covariate shift를 감소시키는 방법을 찾았다. 훈련 과정동안 layer의 입력값 x의 distribution을 고정하면서 훈련 속도를 개선했다. network 훈련의 수렴 속도는 입력값이 whitened되면 더 빠르다고 알려져 있다. -  즉, 평균 0과 분산 1을 이용해 선형 변환하며 decorrelate한다(-> 의미를 잘 모르겠다). 각 layer는 이전 층에서 제공된 입력값을 관측할 때 각 층 입력값에 동일한 whitening을 달성하는 것은 유익하다. 각 층의 입력을 whitening 함으로써 internal covariate shift의 나쁜 영향을 제거할 수 있는 입력의 고정된 distribution을 달성하는데 한 단계 나아갈 수 있다.

We could consider whitening activations at every training step or at some interval, either by modifying the
network directly or by changing the parameters of the
optimization algorithm to depend on the network activation values (Wiesler et al., 2014; Raiko et al., 2012;
Povey et al., 2014; Desjardins & Kavukcuoglu). However, if these modifications are interspersed with the optimization steps, then the gradient descent step may attempt to update the parameters in a way that requires
the normalization to be updated, which reduces the effect of the gradient step. For example, consider a layer
with the input u that adds the learned bias b, and normalizes the result by subtracting the mean of the activation
computed over the training data: $\hat{x} = x − E[x]$ where
$x = u + b, X = {x_{1...N }}$ is the set of values of x over
the training set, and $E[x] = \frac{1}{N}∑_{i=1}^{N} x_i$
. 
If a gradient
descent step ignores the dependence of E[x] on b, then it
will update $b ← b + ∆b$, where $∆b ∝ −∂ℓ/∂\hat{x}$. Then
$u + (b + ∆b) − E[u + (b + ∆b)] = u + b − E[u + b]$.
Thus, the combination of the update to b and subsequent
change in normalization led to no change in the output
of the layer nor, consequently, the loss. As the training
continues, b will grow indefinitely while the loss remains
fixed. This problem can get worse if the normalization not
only centers but also scales the activations. We have observed this empirically in initial experiments, where the
model blows up when the normalization parameters are
computed outside the gradient descent step.

network activation 값에 따라 network를 직접 수정하거나 optimization 알고리즘의 파라미터를 변경하므로써 whitening activation을 매번 training 단계마다 혹은 몇 개의 간격마다 수행할지 고민했다. 그러나 이 modification들이 optimization 단계에 배치되면 gradient descent 단계는 gradient step의 영향을 줄이는 normalization을 update하는 것을 요구하는 방식으로 파라미터를 갱신할 것이다. 예를 들어, b(학습된 편향)를 더하는 u(input)을 가지며 training data에서 계산된 activation의 평균을 결과에 빼줘서 normalize한 layer를 생각해보자. 만약 gradient descent step이 b에 대한 E[x]의 의존도를 무시한다면 이것은 b ← b + ∆b로 갱신된다. 그러므로 u + (b + ∆b) − E[u + (b + ∆b)] = u + b − E[u + b]이 된다. 따라서 b에 대한 업데이트와 정규화의 후속 변경의 combination은 레이어의 출력뿐만 아니라 loss 값 역시 변경하지 않는다. 훈련이 계속되면 b는 손실이 고정된 상태로 무한히 커질 것이다. 이 문제는 정규화가 activation을 중심으로 할 뿐만 아니라 규모를 조정하는 경우 더 악화될 수 있다. 우리는 gradient descent 단계 밖에서 계산된 정규화 파라미터가 계산될 때 model이 폭발했던 초기 실험에서 이를 경험적으로 관찰했다.

The issue with the above approach is that the gradient
descent optimization does not take into account the fact
that the normalization takes place. To address this issue,
we would like to ensure that, for any parameter values,
the network always produces activations with the desired
distribution. Doing so would allow the gradient of the
loss with respect to the model parameters to account for
the normalization, and for its dependence on the model
parameters Θ. Let again x be a layer input, treated as a vector, and X be the set of these inputs over the training
data set. The normalization can then be written as a transformation

$\hat{x}=Norm(x, X)$

which depends not only on the given training example x
but on all examples X – each of which depends on Θ if
x is generated by another layer. For backpropagation, we
would need to compute the Jacobians

$\frac{∂Norm(x, X)}{∂x}
and \frac{∂Norm(x, X)}{∂X}$

ignoring the latter term would lead to the explosion described above. Within this framework, whitening the layer
inputs is expensive, as it requires computing the covariance matrix Cov[x] = Ex∈X [xxT
] − E[x]E[x]T
and its
inverse square root, to produce the whitened activations
Cov[x]−1/2
(x − E[x]), as well as the derivatives of these
transforms for backpropagation. This motivates us to seek
an alternative that performs input normalization in a way
that is differentiable and does not require the analysis of
the entire training set after every parameter update.
Some of the previous approaches (e.g.
(Lyu & Simoncelli, 2008)) use statistics computed
over a single training example, or, in the case of image
networks, over different feature maps at a given location.
However, this changes the representation ability of a
network by discarding the absolute scale of activations.
We want to a preserve the information in the network, by
normalizing the activations in a training example relative
to the statistics of the entire training data.

위 접근 방식의 문제는 gradient descent optimization이 normalization이 일어난다는 사실을 고려하지 못한다는 것이다. 이 문제를 해결하기 위해 우리는 모든 parameter 값들에 대해 network가 항상 희망하는 distribution을 가진 activation을 생성한다고 보장하고 싶다. 이렇게 하는 것은 model parameter에 대한 loss의 gradient가 normalization과 model parameter $Θ$의 dependence를 처리할 수 있게 해준다. vector로 취급되는 x가 다시 층의 input이 되게 하고 X가 전체 training dataset에 대한 입력들의 dataset이라고 하자. 그러면 normalization은 transformation 처럼 $\hat{x} = Norm(x, X)$로 쓸 수 있다. (주어진 training example 뿐만 아니라 모든 샘플 X에도 의존한다.) backpropagation을 위해 우리는 아래의 Jacobians를 계산해야한다. (이전에 서술한 폭발 문제는 뒷 부분을 무시한 것이다.) 이러한 framework는 covariance matrix를 계산해야하기 때문에 비용이 크다. (이후 생략) 

이후 부분은 ppt 자료에 정리한 내용을 참고하자.